In [2]:
from utils import extract_features_for_count_threshold, extract_features_for_dist_threshold
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
from utils import get_matrix_features
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')


# for count threshold


In [2]:
import pandas as pd

threshold_mode = 'count'
num_nn = 4

# colony 0
colony0_segmentation_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_segmentation_T0_to_T146_trimmed.h5'
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'

colony0_features = extract_features_for_count_threshold(
    colony0_segmentation_path, num_nn=num_nn)
colony0_features['colony'] = [0 for i in range(len(colony0_features))]
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_features.to_csv(os.path.join(
    path_to_data, 'colony0_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]
colony0_matrix_features = get_matrix_features(
    colony0_features, colony0_lineage_gt)
# colony0_matrix_features.to_csv(os.path.join(path_to_data, 'matrix_features',
#                                'colony0_matrix_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))


[16:19:13] WARNING: ../src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.


In [ ]:
colony0_features.to_csv(os.path.join(
    path_to_data, 'colony0_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))


In [ ]:
# colony 1 to 5
# load the 5 colony ground truth
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)


# extract features for each colony and save them to a csv file. This takes a while. But after this, we only need to load the csv files. every time.
colony_candidate_features = [0, 0, 0, 0, 0, 0]
args = {"nn_threshold": 8.0, "flexible_nn_threshold": True,
        "num_frames_refractory": 0, "num_frames": 4, "bud_distance_max": 10}
for i in [1, 2, 3, 4, 5]:
    print("finding features for colony {}".format(i))
    colony_candidate_features[i-1] = extract_features_for_count_threshold(os.path.join(
        path_to_colonies, 'colony00{}_segmentation.h5'.format(i)), num_nn=num_nn)
    colony_candidate_features[i-1]['colony'] = i
    # remove rows with time_id = 0
    colony_candidate_features[i-1] = colony_candidate_features[i -
                                                               1][colony_candidate_features[i-1].time_id != 0]
    # remove rows with bud_id == candid_id (this shouldn't happen in theory)
    colony_candidate_features[i-1] = colony_candidate_features[i-1][colony_candidate_features[i-1].bud_id !=
                                                                    colony_candidate_features[i-1].candid_id]
    # save to csv
    colony_candidate_features[i-1].to_csv(os.path.join(
        path_to_colonies, 'colony00{}_candidate_features_{}_{}_nn.csv'.format(i, threshold_mode, num_nn)), index=False)


colonies_features = colony_candidate_features[0]
for i in [1, 2, 3, 4]:
    temp_colony = colony_candidate_features[i]
    colonies_features = pd.concat([colonies_features, temp_colony])
colonies_features = colonies_features.reset_index(drop=True)

colonies_matrix_features = get_matrix_features(colonies_features, colonies_gt)
# colonies_matrix_features.to_csv(os.path.join(path_to_data, 'matrix_features',
#                                 'colony1to5_matrix_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))


# for external data


In [15]:

threshold_mode = 'count'
num_nn = 4

path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'
external_subset_segmentation_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/segmentation2/stacked/mask_stack _edited.h5' 
external_subset_lineage_GT_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited.csv'

# create features for the external subset
external_subset_features = extract_features_for_count_threshold(
    external_subset_segmentation_path, num_nn=num_nn)
external_subset_features['colony'] = [10 for i in range(len(external_subset_features))]
external_subset_features.to_csv(os.path.join(
    path_to_features, 'external_subset_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))

# read the ground truth
external_subset_lineage_gt = pd.read_csv(external_subset_lineage_GT_path).rename(
    columns={'# parent_id': 'parent_GT'})
external_subset_lineage_gt['colony'] = [10 for i in range(len(external_subset_lineage_gt))]

# create matrix features for the external subset
external_subset_matrix_features = get_matrix_features(
    external_subset_features, external_subset_lineage_gt)


[18:48:49] WARNING: /croot/xgboost-split_1675457761144/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
Not enough frames for bud 311 at time 119. Only 1 frames available.
Not enough frames for bud 312 at time 119. Only 1 frames available.
Not enough frames for bud 313 at time 119. Only 1 frames available.
Not enough frames for bud 314 at time 119. Only 1 frames available.
Not enough frames for bud 315 at time 119. Only 1 frames available.
Not enough frames for bud 316 at time 119. Only 1 frames available.
Not enough frames for bud 317 at time 119. Only 1 frames available.
Not enough frames for bud 318 at time 119. Only 1 frames available.
Not enough frames for bud 319 at time 119. Only 1 frames available.
Not enough frames for bud 320 at time 119. Only 1 frames available.
Not enough frames for bud 321 at time 119. Only 1 frames available.
Not enough

In [17]:
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'

external_subset_lineage_gt.to_csv(external_subset_lineage_GT_path_edited)

In [20]:
external_subset_lineage_gt.loc[(external_subset_lineage_gt['parent_GT'] > 0) & (external_subset_lineage_gt['parent_GT'] < 1000000), 'parent_GT'] = external_subset_lineage_gt.loc[(external_subset_lineage_gt['parent_GT'] > 0) & (external_subset_lineage_gt['time_index'] < 119)]

In [16]:
external_subset_matrix_features

,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
3,1,4,6,10,"[[1.0, 4.0, 1.4643841598820833, 1.533490694990...","[1, 2, 3, -3]",0
4,2,5,8,10,"[[1.0, 2.8284271247461903, 1.8341048520519745,...","[2, 3, 1, 4]",0
5,3,6,9,10,"[[2.0, 2.23606797749979, 0.9843862591629123, 1...","[3, 5, 2, 1]",0
6,1,7,18,10,"[[1.0, 2.8284271247461903, 1.7695565443728838,...","[1, 4, 2, 3]",0
7,2,8,21,10,"[[1.0, 2.0, 1.8103577502210595, 1.819063025188...","[1, 2, 5, 3]",1
...,...,...,...,...,...,...,...
298,30,299,118,10,"[[1.0, 2.0, 2.0916317995188516, 2.010341104017...","[30, 52, 198, 239]",0
300,160,301,118,10,"[[1.0, 2.0, 0.5943675582008001, 0.662130820442...","[38, 160, 27, 90]",1
301,27,302,118,10,"[[1.0, 4.0, 4.516708573096808, 4.5093907953453...","[27, 90, 160, 38]",0
304,26,305,118,10,"[[1.0, 2.23606797749979, 0.9151683004515565, 1...","[26, 158, 237, 108]",0


In [13]:
external_subset_features

,bud_id,candid_id,time_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,colony
0,1,1,0,0.000000,0.000000,0.000000,0.136735,0.472682,0.062055,0.416098,0.062055,-0.073019,0.085530,0
0,1,2,0,1.414214,2.000000,0.018181,0.039193,0.180781,1.334069,0.036880,1.456874,0.001212,1.455263,0
0,1,3,0,2.828427,2.828427,0.428816,0.305553,0.901870,1.517028,0.051967,0.581480,-0.009488,0.559341,0
0,2,2,0,0.000000,0.000000,0.000000,-0.008218,0.029242,0.181369,0.379370,0.181369,0.160461,0.815378,0
0,2,1,0,1.414214,2.000000,0.018181,0.063909,0.147795,1.465211,0.014948,1.325732,0.026675,1.422776,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,309,125,118,69.771054,69.771054,-8.809561,-9.757378,-6.285668,1.459236,0.689896,1.093371,-0.189290,0.904081,0
0,310,116,118,55.027266,55.027266,-27.764721,-27.863648,-26.778800,0.921513,0.577551,1.231598,-0.316615,0.914983,0
0,310,125,118,59.008474,59.008474,-12.994342,-12.871260,-8.130301,0.977370,0.519918,1.517390,-0.199940,1.317450,0
0,310,220,118,62.425956,71.505245,11.258358,13.288559,8.237581,1.304395,0.709578,0.926194,0.388442,1.314636,0


In [21]:
external_subset_matrix_features

,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
3,1,4,6,10,"[[1.0, 4.0, 1.4643841598820833, 1.533490694990...","[1, 2, 3, -3]",0
4,2,5,8,10,"[[1.0, 2.8284271247461903, 1.8341048520519745,...","[2, 3, 1, 4]",0
5,3,6,9,10,"[[2.0, 2.23606797749979, 0.9843862591629123, 1...","[3, 5, 2, 1]",0
6,1,7,18,10,"[[1.0, 2.8284271247461903, 1.7695565443728838,...","[1, 4, 2, 3]",0
7,2,8,21,10,"[[1.0, 2.0, 1.8103577502210595, 1.819063025188...","[1, 2, 5, 3]",1
...,...,...,...,...,...,...,...
298,30,299,118,10,"[[1.0, 2.0, 2.0916317995188516, 2.010341104017...","[30, 52, 198, 239]",0
300,160,301,118,10,"[[1.0, 2.0, 0.5943675582008001, 0.662130820442...","[38, 160, 27, 90]",1
301,27,302,118,10,"[[1.0, 4.0, 4.516708573096808, 4.5093907953453...","[27, 90, 160, 38]",0
304,26,305,118,10,"[[1.0, 2.23606797749979, 0.9151683004515565, 1...","[26, 158, 237, 108]",0
